In [1]:

#check what version of python you're using - I'm using 3.7.3
from platform import python_version
print(python_version())


# In[3]:

#importing libraries
from astropy.io import fits
from astropy.convolution import convolve, Gaussian2DKernel
from astropy.wcs import WCS
from reproject import reproject_exact  #a package that can be added to astropy using anaconda or pip (see their docs pg)
from reproject import reproject_interp

import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys


3.7.4


In [2]:
# #finding the path to every fits images in a directory
def im_name_finder(path, file_type):
    #Using glob (it's a unix command similar to ls)
    #WARNING: using recursive=True...depending how many images you use this could be very slow, it's recommended not to have too many subfolders
    #if needed, some example code is commented towards the latter half of this code that could help make an alternative
    all_names = glob.glob(path, recursive=True)

    #IMPORTANT: Using "fit" here because it is inclusive of both fits and FIT...some files end in "FIT" and need to be included
    #using s.lower() include uppercase names
    im_names = [s for s in all_names if 'fit' in s.lower()]

    return im_names


# In[27]:

#setting up a new fits file to be saved and viewed in DS9
#primarily to save the image we reprojected, but can also be used to save the convolved images
def fits_saver(array, wcs_header, name, save_path):
    '''
    array is a 2d array of data - could be from reprojecting one image onto another or from convolution
    wcs_header is a header containing the wcs coords of the image that we projected onto or of the orig image (if from the convolution)
    name is the path to some image you're using. It will get string split at the / character, and the func only takes the last element of that splitting
    save_path is the folder you want to save to...recommended to also add something to the start of the images names to make it clear what you did to them (e.g. 'Regridded/regrid_')
    '''

    #creating a new file and adding the reprojected array of data as well as the WCS that we projected onto
    hdu_new = fits.PrimaryHDU(array, header=wcs_header)
    hdul = fits.HDUList([hdu_new])

    #saving the file
    new_filename = name.split('/')[-1]  #grabs the file name we were using from before
    hdul.writeto(save_path+new_filename, overwrite=True)

    return (save_path+new_filename)



In [5]:

# In[21]:
#declaring noise: measured from the RMS of a region of empty sky while opening the image in DS9
#these are newly taken from the HH 6 images...
# noise_126 = 0.0465246
# noise_128 = 0.04733
# noise_164 = 0.0561296

#from the regridded images...
noise_126 = 0.216037
noise_128 = 0.258513
noise_164 = 0.39321
noise_halpha = 0.302712
#converting units for reference
#hdu1_conv_scaled = hdu1_conv_scaled * hdu1_fnu / 1e6 #converting to MJy
#hdu1_conv_scaled = hdu1_conv_scaled / hdu1_pix**2. * 4.25e10 #dividing out sr, D001SCAL is key for pixel size in arcsec



#EX: grabbing all the fits image paths in a directory, so they can be looped through and their data opened
#set your path to some directory with images (the images can be in subdirectories)
#using ** will grab all files even in subdirectories...WARNING this will take longer
path = '../../../ngc1333_fits/'
im_names_hub_dash = im_name_finder(path+'*', 'fit')
im_names_hub_dash = [i.replace('\\', '/') for i in im_names_hub_dash]
im_names_hub = [path+'126_image.fits', path+'128_image.fits', path+'164_image.fits']


# In[28]:

#Minimal loop through all the images, including a try/except in case an image is faulty for whatever reason
#IMPORTANT: A more detailed example between two single images is at the end of this code with many more comments

#First, we need to setup the image we're projecting ONTO
#This will be the same no matter the loop, so only need to do this once
#low_res = [x for x in im_names_spitz if 'n1333_lh_3_SiII_' in x][0]  #finding the lowest res image - LH and long lambda, so [SiII]
#hdu1 = fits.open(low_res)[0]
low_res = im_names_hub_dash[0]   #Could also let this be a hubble image...if so, see the hubble loop below for how to setup grabbing the data, header, pixel conversion from the hdu
hdu1 = fits.open(low_res)

#hdu1_pix = hdu1.header['CDELT2'] #the pixel size in degrees, CDELT is the keyword for Spitzer images
#hdu1_pix_torad = hdu1_pix * np.pi / 180.
hdu1_pix = hdu1[0].header['D001SCAL'] #same as above line, but D001SCAL is the keyword for Hubble images
hdu1_pix_torad = hdu1_pix / 206265.
hdu1_fnu = hdu1[0].header['PHOTFNU']

low_res = im_names_hub[0]
hdu1 = fits.open(low_res)
hdu1_data = hdu1[0].data
hdu1_header = hdu1[0].header

#converting noise units
noise_126 = noise_126 * hdu1_fnu / 1e6


In [7]:
for name in [im_names_hub[1]]:
    hdu2 = fits.open(name)  #Image we're reprojecting...no index yet since some useful data split between headers
    hdu2 = fits.open(im_names_hub_dash[2])
    #try:

    #reading in data
    hdu2_pix = hdu2[0].header['D001SCAL'] #same as above line, but D001SCAL is the keyword for Hubble images
    hdu2_pix_torad = hdu2_pix / 206265.
    hdu2_fnu = hdu2[0].header['PHOTFNU']

    hdu2 = fits.open(name)
    hdu2_data = hdu2[0].data
    hdu2_header = hdu2[0].header

    #converting noise units
    noise_128 = noise_128 * hdu2_fnu / 1e6




    #need to do 164 image as well...
    name2 = im_names_hub[2]
    hdu3 = fits.open(im_names_hub_dash[4])
    #try:

    #reading in data
    hdu3_pix = hdu3[0].header['D001SCAL'] #same as above line, but D001SCAL is the keyword for Hubble images
    hdu3_pix_torad = hdu3_pix / 206265.
    hdu3_fnu = hdu3[0].header['PHOTFNU']

    hdu3 = fits.open(name2)
    hdu3_data = hdu3[0].data
    hdu3_header = hdu3[0].header

    #converting noise units
    noise_164 = noise_164 * hdu3_fnu / 1e6



    #reprojection of one hdu using the header (coords and pixels) of another
    #The first input is the path to the file we're reprojecting. The second input is the header of the image we're projecting ONTO
    #you'll need to set the WCS to be that of the header you're basing this off of...ie the header

    #file_start = 'Convolved_Images_Hub/conv_'
    #conv2_path = file_start + name.split('/')[-1]

    w = WCS(hdu1_header)
    wcs_header = w.to_header()
    file_start = '../../Convolved_Images_Hub/conv_'
    hdu1_conv = fits.open(file_start + low_res.split('/')[-1])
    hdu1_conv_scaled = hdu1_conv[0].data * hdu1_pix**2. / 4.25e10


    #para is False for large images (like these hubble ones)
    #output is array (a 2D array of data) and footprint (the footprint from the analysis)
    file_start = '../../Regridded_Hub/regrid_'
    hdu_regrid_hub = fits.open(file_start + name.split('/')[-1])
    array = hdu_regrid_hub[0].data * hdu2_pix**2. / 4.25e10


    hdu_regrid_hub2 = fits.open(file_start + name2.split('/')[-1])
    array2 = hdu_regrid_hub2[0].data * hdu3_pix**2. / 4.25e10


    #now that we have a reprojected hubble image for hdu2 and both are convolved, need to
    #divide one by the other...then can use the same wcs header that we projected onto (hdu1's)!
    #getting rid of nan values
    where_are_NaNs = np.isnan(array)
    array[where_are_NaNs] = 0.

    where_are_NaNs = np.isnan(array2)
    array2[where_are_NaNs] = 0.

    where_are_NaNs = np.isnan(hdu1_conv_scaled)
    hdu1_conv_scaled[where_are_NaNs] = 0.

    #print(perc * np.max(array))
    #array[array < (perc * np.max(array))] = 0.  #only taking values greater than some percent of the maximum


    #scaling by the signal to noise
    perc = 0.8 #this tends to work?? Maybe can change...
    
    array[array < noise_128*perc] = 0.
    array2[array2 < noise_164*perc] = 0.
    hdu1_conv_scaled[hdu1_conv_scaled < noise_126*perc] = 0.

    #getting rid of 0 values
    array[array < 0.] = 0.
    array2[array2 < 0.] = 0.
    hdu1_conv_scaled[hdu1_conv_scaled < 0.] = 0.



#     eps = 8e-1 
#     epslist = [1e-3, 1e-2, 5e-2, 1e-1, 5e-1, 7e-1, 8e-1, 9e-1, 99e-1]
    epslist = [0.0]
    
    for eps in epslist:
        flux01 = hdu1_conv_scaled - eps * array
        flux02 = array2
        data_ratio = np.divide(flux01, flux02, out=np.zeros_like(flux02), where=flux02!=0.) #need to do np.divide to guarantee we get no divide by zero issue...


        # remember to have the right header with the wcs below and that it matches the one we're projecting ONTO
        w = WCS(hdu1_header)
        wcs_header = w.to_header()
        save_path = './regrid_'  #See fits_saver's "save_path" description for explanation
        fits_saver(data_ratio, wcs_header, 'hub_dash_noleakage_ston_'+str(perc)+'.fits', save_path)  #saving the reprojected image

    

sys.exit()



SystemExit: 

C:\Users\Adam2\Anaconda\envs\mypython3\lib\site-packages\IPython\core\interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
path = 'inanna_runs/'
im_names_hub = [path+'nonlin_flam_1_26_mic.fits', path+'nonlin_flam_1_64_mic.fits']

im_126 = im_names_hub[0]   
hdu1 = fits.open(im_126)
hdu1_data = hdu1[0].data
hdu1_header = hdu1[0].header

im_164 = im_names_hub[1]   
hdu2 = fits.open(im_164)
hdu2_data = hdu2[0].data
hdu2_header = hdu2[0].header


data_ratio = np.divide(hdu1_data, hdu2_data, out=np.zeros_like(hdu2_data), where=hdu2_data!=0.) #need to do np.divide to guarantee we get no divide by zero issue...


# remember to have the right header with the wcs below and that it matches the one we're projecting ONTO
w = WCS(hdu1_header)
wcs_header = w.to_header()
save_path = './nonlintest_'  #See fits_saver's "save_path" description for explanation
fits_saver(data_ratio, wcs_header, 'hub_dash_noleakage_ston_-1.fits', save_path)  #saving the reprojected image

    


'./nonlintest_hub_dash_noleakage_ston_-1.fits'